In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [83]:
df = pd.read_csv('fifa21_train.csv')
#print(df.shape)
#print(df.dtypes)
print(df.isna().sum())

ID              0
Name            0
Age             0
Nationality     0
Club           18
               ..
CB              0
RCB             0
RB              0
GK              0
OVA             0
Length: 101, dtype: int64


In [84]:
df.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [85]:
numerical_cols = df._get_numeric_data()
categorical_cols = df.select_dtypes('object')

In [86]:
numerical_cols

,ID,Age,Growth,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,...,GK Reflexes,Total Stats,Base Stats,PAC,SHO,PAS,DRI,DEF,PHY,OVA
0,184383,26,1,258,54,47,43,70,44.0,286,...,6,1682,357,69,51,63,63,51,60,64
1,188044,30,0,365,66,79,76,68,76.0,375,...,16,1961,412,83,75,68,82,33,71,77
2,184431,33,0,336,73,76,34,78,75.0,424,...,3,1925,404,80,77,78,86,27,56,80
3,233796,22,13,242,44,42,58,62,36.0,259,...,12,1527,329,57,44,54,57,57,60,59
4,234799,23,8,249,49,37,61,68,34.0,280,...,15,1664,360,66,44,60,64,60,66,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,25,5,224,62,24,51,59,28.0,248,...,14,1589,347,75,28,56,59,60,69,65
11697,214680,27,4,84,12,14,13,29,16.0,96,...,72,1123,387,72,67,66,72,40,70,70
11698,221489,22,8,286,64,66,51,64,41.0,291,...,16,1603,337,76,60,60,72,26,43,67
11699,146717,26,7,254,64,66,51,73,NaN,284,...,24,1429,400,72,63,70,72,57,66,68


In [87]:
categorical_cols

,Name,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Joined,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,"Jul 1, 2015",...,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1
1,Alan Carvalho,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,"Jan 16, 2015",...,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2
2,S. Giovinco,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,"Jan 31, 2019",...,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2
3,J. Evans,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,"Jul 1, 2016",...,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2
4,Y. Demoncy,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,"Jul 1, 2018",...,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,B. Böðvarsson,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,"6'1""",168lbs,Left,"Feb 8, 2018",...,60+2,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2
11697,G. Gallon,France,ESTAC Troyes,GK,GK,ESTAC Troyes 2019 ~ 2022,"6'1""",174lbs,Right,"Jul 1, 2019",...,26+2,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2
11698,J. Flores,Chile,CD Antofagasta,RM,LM CAM RM,CD Antofagasta 2019 ~ 2024,"5'6""",143lbs,Right,"Jan 23, 2019",...,44+2,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2
11699,Anderson Silva,Brazil,Barnsley,CM,NaN,Barnsley 2010,"6'2""",179lbs,Right,NaN,...,68+0,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0


In [88]:
df['Club'] = df['Club'].fillna('Free Agent')
unique_club = df['Club'].value_counts()
unique_club

Everton           35
Crystal Palace    35
OGC Nice          35
Chelsea           33
Sporting CP       33
                  ..
GIF Sundsvall      1
FC Orenburg        1
Germany            1
LKS Lodz           1
Santo André        1
Name: Club, Length: 886, dtype: int64

In [89]:
df.isna().sum()

ID             0
Name           0
Age            0
Nationality    0
Club           0
              ..
CB             0
RCB            0
RB             0
GK             0
OVA            0
Length: 101, dtype: int64

In [90]:
columns = df.columns
columns

Index(['ID', 'Name', 'Age', 'Nationality', 'Club', 'BP', 'Position',
       'Team & Contract', 'Height', 'Weight',
       ...
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK', 'OVA'],
      dtype='object', length=101)

In [91]:
for col in df.columns:
    print(col)
    #print(len(df[col].unique()))

ID
Name
Age
Nationality
Club
BP
Position
Team & Contract
Height
Weight
foot
Growth
Joined
Loan Date End
Value
Wage
Release Clause
Contract
Attacking
Crossing
Finishing
Heading Accuracy
Short Passing
Volleys
Skill
Dribbling
Curve
FK Accuracy
Long Passing
Ball Control
Movement
Acceleration
Sprint Speed
Agility
Reactions
Balance
Power
Shot Power
Jumping
Stamina
Strength
Long Shots
Mentality
Aggression
Interceptions
Positioning
Vision
Penalties
Composure
Defending
Marking
Standing Tackle
Sliding Tackle
Goalkeeping
GK Diving
GK Handling
GK Kicking
GK Positioning
GK Reflexes
Total Stats
Base Stats
W/F
SM
A/W
D/W
IR
PAC
SHO
PAS
DRI
DEF
PHY
Hits
LS
ST
RS
LW
LF
CF
RF
RW
LAM
CAM
RAM
LM
LCM
CM
RCM
RM
LWB
LDM
CDM
RDM
RWB
LB
LCB
CB
RCB
RB
GK
OVA


In [92]:
df = df.drop(['Age','Nationality','Club','Team & Contract','Height','Weight','foot','Growth','Joined','Loan Date End','Value','Wage','Release Clause','Contract','W/F','SM','A/W','D/W','Hits','Position','ID','Name'], axis=1)

In [93]:
df.shape

(11701, 79)

In [94]:
df.columns

Index(['BP', 'Attacking', 'Crossing', 'Finishing', 'Heading Accuracy',
       'Short Passing', 'Volleys', 'Skill', 'Dribbling', 'Curve',
       'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'LS',
       'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM',
       'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
       'LCB', 'CB', 'RCB', 'RB', 'GK', 'OVA'],
      dtype='object')

In [95]:
duplicate_rows = df.duplicated()
duplicate_rows.unique()

array([False])

In [96]:
def remove_plus_and_after(value):
    if isinstance(value, str):
        return value.split('+')[0]
    return value

df = df.applymap(remove_plus_and_after)
df

,BP,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,CM,258,54,47,43,70,44.0,286,61,44.0,...,59,59,59,58,54,54,54,58,15,64
1,ST,365,66,79,76,68,76.0,375,83,78.0,...,53,53,57,53,48,48,48,53,18,77
2,CAM,336,73,76,34,78,75.0,424,85,89.0,...,56,56,59,53,41,41,41,53,12,80
3,CDM,242,44,42,58,62,36.0,259,54,41.0,...,58,58,56,57,58,58,58,57,14,59
4,CDM,249,49,37,61,68,34.0,280,64,44.0,...,64,64,64,63,61,61,61,63,15,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,LB,224,62,24,51,59,28.0,248,57,49.0,...,60,60,63,63,61,61,61,63,16,65
11697,GK,84,12,14,13,29,16.0,96,17,15.0,...,26,26,25,24,26,26,26,24,69,70
11698,RM,286,64,66,51,64,41.0,291,71,57.0,...,44,44,49,45,35,35,35,45,17,67
11699,CM,254,64,66,51,73,NaN,284,74,NaN,...,68,68,66,64,60,60,60,64,25,68


In [97]:
column_means = df.mean()
df = df.fillna(column_means)
df

/var/folders/kd/gp0d9cx1091g1l19n5h72dhr0000gn/T/ipykernel_4302/2185361014.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()


,BP,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,CM,258,54,47,43,70,44.000000,286,61,44.000000,...,59,59,59,58,54,54,54,58,15,64
1,ST,365,66,79,76,68,76.000000,375,83,78.000000,...,53,53,57,53,48,48,48,53,18,77
2,CAM,336,73,76,34,78,75.000000,424,85,89.000000,...,56,56,59,53,41,41,41,53,12,80
3,CDM,242,44,42,58,62,36.000000,259,54,41.000000,...,58,58,56,57,58,58,58,57,14,59
4,CDM,249,49,37,61,68,34.000000,280,64,44.000000,...,64,64,64,63,61,61,61,63,15,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,LB,224,62,24,51,59,28.000000,248,57,49.000000,...,60,60,63,63,61,61,61,63,16,65
11697,GK,84,12,14,13,29,16.000000,96,17,15.000000,...,26,26,25,24,26,26,26,24,69,70
11698,RM,286,64,66,51,64,41.000000,291,71,57.000000,...,44,44,49,45,35,35,35,45,17,67
11699,CM,254,64,66,51,73,45.067324,284,74,49.617839,...,68,68,66,64,60,60,60,64,25,68


In [98]:
df = df.round(0)
df

,BP,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,CM,258,54,47,43,70,44.0,286,61,44.0,...,59,59,59,58,54,54,54,58,15,64
1,ST,365,66,79,76,68,76.0,375,83,78.0,...,53,53,57,53,48,48,48,53,18,77
2,CAM,336,73,76,34,78,75.0,424,85,89.0,...,56,56,59,53,41,41,41,53,12,80
3,CDM,242,44,42,58,62,36.0,259,54,41.0,...,58,58,56,57,58,58,58,57,14,59
4,CDM,249,49,37,61,68,34.0,280,64,44.0,...,64,64,64,63,61,61,61,63,15,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,LB,224,62,24,51,59,28.0,248,57,49.0,...,60,60,63,63,61,61,61,63,16,65
11697,GK,84,12,14,13,29,16.0,96,17,15.0,...,26,26,25,24,26,26,26,24,69,70
11698,RM,286,64,66,51,64,41.0,291,71,57.0,...,44,44,49,45,35,35,35,45,17,67
11699,CM,254,64,66,51,73,45.0,284,74,50.0,...,68,68,66,64,60,60,60,64,25,68


In [99]:
X = df.drop(['OVA'], axis=1)
X

,BP,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,CM,258,54,47,43,70,44.0,286,61,44.0,...,59,59,59,59,58,54,54,54,58,15
1,ST,365,66,79,76,68,76.0,375,83,78.0,...,53,53,53,57,53,48,48,48,53,18
2,CAM,336,73,76,34,78,75.0,424,85,89.0,...,56,56,56,59,53,41,41,41,53,12
3,CDM,242,44,42,58,62,36.0,259,54,41.0,...,58,58,58,56,57,58,58,58,57,14
4,CDM,249,49,37,61,68,34.0,280,64,44.0,...,64,64,64,64,63,61,61,61,63,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,LB,224,62,24,51,59,28.0,248,57,49.0,...,60,60,60,63,63,61,61,61,63,16
11697,GK,84,12,14,13,29,16.0,96,17,15.0,...,26,26,26,25,24,26,26,26,24,69
11698,RM,286,64,66,51,64,41.0,291,71,57.0,...,44,44,44,49,45,35,35,35,45,17
11699,CM,254,64,66,51,73,45.0,284,74,50.0,...,68,68,68,66,64,60,60,60,64,25


In [100]:
y = df['OVA']
y

0        64
1        77
2        80
3        59
4        65
         ..
11696    65
11697    70
11698    67
11699    68
11700    64
Name: OVA, Length: 11701, dtype: int64

In [101]:
X_cat = df.select_dtypes('object')
X_num = df._get_numeric_data() 

In [102]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(X_cat)

cols = encoder.get_feature_names_out(input_features=X_cat.columns)

X_cat_encode = pd.DataFrame(encoder.transform(X_cat).toarray(),columns=cols)

X_cat_encode.head()

,BP_CB,BP_CDM,BP_CF,BP_CM,BP_GK,BP_LB,BP_LM,BP_LW,BP_LWB,BP_RB,...,GK_78,GK_79,GK_80,GK_81,GK_82,GK_83,GK_84,GK_85,GK_88,GK_9
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
X_cat=pd.get_dummies(X_cat, drop_first=True)
X=pd.concat([X_num, X_cat], axis=1)
X.head()

,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,...,GK_78,GK_79,GK_80,GK_81,GK_82,GK_83,GK_84,GK_85,GK_88,GK_9
0,258,54,47,43,70,44.0,286,61,44.0,55,...,0,0,0,0,0,0,0,0,0,0
1,365,66,79,76,68,76.0,375,83,78.0,72,...,0,0,0,0,0,0,0,0,0,0
2,336,73,76,34,78,75.0,424,85,89.0,91,...,0,0,0,0,0,0,0,0,0,0
3,242,44,42,58,62,36.0,259,54,41.0,46,...,0,0,0,0,0,0,0,0,0,0
4,249,49,37,61,68,34.0,280,64,44.0,45,...,0,0,0,0,0,0,0,0,0,0


In [104]:
# Normalizing the data

In [105]:
from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different

transformer = MinMaxScaler().fit(X_num)
x_normalized = transformer.transform(X_num)
print(x_normalized.shape)
x_normalized
pd.DataFrame(x_normalized, columns=X_num.columns)

(11701, 50)


,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,...,GK Reflexes,Total Stats,Base Stats,PAC,SHO,PAS,DRI,DEF,PHY,OVA
0,0.556931,0.545455,0.478261,0.431818,0.720930,0.465116,0.569087,0.615385,0.444444,0.561798,...,0.056180,0.604577,0.492366,0.614286,0.447368,0.558824,0.522388,0.506494,0.500000,0.472727
1,0.821782,0.681818,0.826087,0.806818,0.697674,0.837209,0.777518,0.857143,0.822222,0.752809,...,0.168539,0.781945,0.702290,0.814286,0.763158,0.632353,0.805970,0.272727,0.666667,0.709091
2,0.750000,0.761364,0.793478,0.329545,0.813953,0.825581,0.892272,0.879121,0.944444,0.966292,...,0.022472,0.759059,0.671756,0.771429,0.789474,0.779412,0.865672,0.194805,0.439394,0.763636
3,0.517327,0.431818,0.423913,0.602273,0.627907,0.372093,0.505855,0.538462,0.411111,0.460674,...,0.123596,0.506039,0.385496,0.442857,0.355263,0.426471,0.432836,0.584416,0.500000,0.381818
4,0.534653,0.488636,0.369565,0.636364,0.697674,0.348837,0.555035,0.648352,0.444444,0.449438,...,0.157303,0.593134,0.503817,0.571429,0.355263,0.514706,0.537313,0.623377,0.590909,0.490909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,0.472772,0.636364,0.228261,0.522727,0.593023,0.279070,0.480094,0.571429,0.500000,0.269663,...,0.146067,0.545455,0.454198,0.700000,0.144737,0.455882,0.462687,0.623377,0.636364,0.490909
11697,0.126238,0.068182,0.119565,0.090909,0.244186,0.139535,0.124122,0.131868,0.122222,0.157303,...,0.797753,0.249205,0.606870,0.657143,0.657895,0.602941,0.656716,0.363636,0.651515,0.581818
11698,0.626238,0.659091,0.684783,0.522727,0.651163,0.430233,0.580796,0.725275,0.588889,0.370787,...,0.168539,0.554355,0.416031,0.714286,0.565789,0.514706,0.656716,0.181818,0.242424,0.527273
11699,0.547030,0.659091,0.684783,0.522727,0.755814,0.476744,0.564403,0.758242,0.511111,0.651685,...,0.258427,0.443738,0.656489,0.657143,0.605263,0.661765,0.656716,0.584416,0.590909,0.545455


In [106]:
round(pd.DataFrame(x_normalized, columns=X_num.columns),1)

,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,...,GK Reflexes,Total Stats,Base Stats,PAC,SHO,PAS,DRI,DEF,PHY,OVA
0,0.6,0.5,0.5,0.4,0.7,0.5,0.6,0.6,0.4,0.6,...,0.1,0.6,0.5,0.6,0.4,0.6,0.5,0.5,0.5,0.5
1,0.8,0.7,0.8,0.8,0.7,0.8,0.8,0.9,0.8,0.8,...,0.2,0.8,0.7,0.8,0.8,0.6,0.8,0.3,0.7,0.7
2,0.8,0.8,0.8,0.3,0.8,0.8,0.9,0.9,0.9,1.0,...,0.0,0.8,0.7,0.8,0.8,0.8,0.9,0.2,0.4,0.8
3,0.5,0.4,0.4,0.6,0.6,0.4,0.5,0.5,0.4,0.5,...,0.1,0.5,0.4,0.4,0.4,0.4,0.4,0.6,0.5,0.4
4,0.5,0.5,0.4,0.6,0.7,0.3,0.6,0.6,0.4,0.4,...,0.2,0.6,0.5,0.6,0.4,0.5,0.5,0.6,0.6,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,0.5,0.6,0.2,0.5,0.6,0.3,0.5,0.6,0.5,0.3,...,0.1,0.5,0.5,0.7,0.1,0.5,0.5,0.6,0.6,0.5
11697,0.1,0.1,0.1,0.1,0.2,0.1,0.1,0.1,0.1,0.2,...,0.8,0.2,0.6,0.7,0.7,0.6,0.7,0.4,0.7,0.6
11698,0.6,0.7,0.7,0.5,0.7,0.4,0.6,0.7,0.6,0.4,...,0.2,0.6,0.4,0.7,0.6,0.5,0.7,0.2,0.2,0.5
11699,0.5,0.7,0.7,0.5,0.8,0.5,0.6,0.8,0.5,0.7,...,0.3,0.4,0.7,0.7,0.6,0.7,0.7,0.6,0.6,0.5


In [111]:
X = x_normalized

In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9360, 50)
(2341, 50)
(9360,)
(2341,)


In [113]:
y_train.head()

1775    62
2455    74
93      69
8952    55
7810    79
Name: OVA, dtype: int64

In [114]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [115]:
correlations_matrix = X.corr()
correlations_matrix

AttributeError: 'numpy.ndarray' object has no attribute 'corr'

In [ ]:
#the last thing should be create a def with all the code we used
